<a href="https://colab.research.google.com/github/shrigoa/random/blob/main/Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem:
1. Manage internet bandwidth usage on a ship.

Important data:
1. Access to two communication links each contracted with 15TB of bandwidth for a month, thus, total of 30TB.

Goals:
1. Optimal usage: Bandwidth should be consumed only when required and minimized when not needed.
2.a. Fairness Across Time: Quality of Experience (QoE) should be consistent across different timeslots.
2.b. Significant variations — such as excellent QoE in some periods and poor QoE in others — must be inimized to avoid service quality alarms.
2.c. Demand met percentage is the QoE. It should not vary much.

Given Data:
1. Per hour demand data for upstream and downstream traffic
- Variations of 20-25% are possible
- Columns: Time slots, downstream demand, upstream demand
2. Ship arrival at ports
- Variation of 1-2 hours earlier or later.
- Demand drops 50% at port compared to sea.
- Columns: Port, Arrival Time, Departure Time
3. Link Downtime
- One of the links will be unavailable on certain days.
- This downtime must be configurable in the code.
- During downtime, the algorithm must reserve enough bandwidth on the remaining active link to maintain stable QoE and avoid service degradation.
4. Consumption Limits:
- Each link supports maximum bandwidth of 500Mbps at any given point.
- Bandwidth planning is done in 5 min time-slots.
- Bandwidth volume (e.g., 1 GB in 5 minutes) will internally be converted into rate for enforcement.
5. End of the month requirement:
- Allocated 30TB should be completely used. (CONSTRAINT)
- No under/over usage allowed.
6. Objective:
- Demand met percentage is the QoE (OBJECTIVE FUNCTION).
- Needs to be as high as possible.

7. Decision Variables






